In [1]:
%cd d:\Laptop_Projects\LLMEval-1

d:\Laptop_Projects\LLMEval-1


In [2]:
from collections import Counter
import numpy as np
from summac.model_guardrails import NERInaccuracyPenalty
import nltk
import spacy
import json
import pandas as pd
import string
import tfidf_matcher as tm
from fuzzywuzzy import fuzz
from server.metrics.faithfulness_helper import FaithfulnessHelper,get_quartiles_faithfulness
from server.metrics.stylistic import FaithfulnessEvaluator

In [3]:
def save_json(data, filepath=r'new_data.json'):
    with open(filepath, 'w') as fp:
        json.dump(data, fp, indent=4)

Load data, all class objects and meta data

In [4]:
data = json.load(open(r'D:\Laptop_Projects\LLMEval-1\preprocess\experiments\data\Faithfulness_scores.json'))
meta_data = json.load(open(r'D:\Laptop_Projects\LLMEval-1\preprocess\experiments\data\meta_data.json'))
df = pd.DataFrame(data)
fh = FaithfulnessHelper()
md = meta_data["faithfulness"]["ranges"]
ranges = []
for range in md:
    ranges.append([range[0],range[1]])
fe = FaithfulnessEvaluator(ranges) #Pass empty string if you dont have Faithfulness_test.json

In [5]:
import spacy
nlp = spacy.load("en_core_web_sm")
import en_core_web_sm
nlp = en_core_web_sm.load()
doc = nlp("MPs have criticized the legal system for often disadvantaging divorced parents seeking access to their children, stating that parents with custody can exploit delays to prevent former partners from gaining access. While rejecting the idea of a legal presumption of equal access, they called for more rigorous enforcement of court orders and a change in the law to specifically consider the importance of sustaining the relationship between the child and the non-resident parent in contested cases. They also emphasized the need for professional mediation and negotiation to be prioritized over court involvement in resolving these sensitive family matters.")
{(ent.text.strip(), ent.label_) for ent in doc.ents}

set()

In [6]:
fuzz.token_set_ratio("Committee","five month")

21

New Instance example

In [7]:
article_text = '"MPs criticise child access system\n \n Divorced parents seeking access to their children are often disadvantaged by the legal system, MPs have said.\n \n The Constitutional Affairs Select Committee said parents with custody could exploit delays in the system to stop former partners gaining access. Courts should be used as a last resort, but where they are, their orders should be enforced more rigorously, MPs said. But they rejected the claim made by some campaign groups that there should be a legal presumption of equal access.\n \n Currently the presumption is that the interests of the child are paramount. \"An arbitrary template imposed on all families, whatever the needs of the child, would relegate the welfare of individual children to a secondary position,\" the MPs said.\n \n They said the law should be changed to require family courts specifically to take account of the importance of sustaining the relationship between the child and the non-resident parent in contested cases. This would \"reassert the rights of non-resident parents to contact with their children, as well as the rights of children to contact with both their parents, while maintaining sufficient flexibility to cope with issues of safety\", they said. Delays in court hearings and the inability to effectively enforce court orders allowed \"a new status quo arrangement for the children to become established by default\", they said.\n \n \"Although the courts rigorously avoid conscious bias, there are considerable grounds for accepting that non-resident parents are frequently disadvantaged by the system as it is administered at present. \"Given the strong animosity between the parties which is common in contested family cases, we find it hard to believe that tactical delay is not sometimes used to the advantage of resident parents.\" Committee chairman Alan Beith said the five-month inquiry had been a \"complex and emotive\" one. \"At the moment, far too many contact and residency cases are being dealt with by the courts when they could be better resolved through professional mediation and negotiation,\" he said. \"This situation has to change. The court system should only be used as a last resort, where mediation and negotiation have completely broken down or where issues of abuse or domestic violence need to be dealt with. \"This will help to reduce delays and improve the lives of many children across the country.\"\n"'
summ ='MPs have criticized the legal system for often disadvantaging divorced parents seeking access to their children, stating that parents with custody can exploit delays to prevent former partners from gaining access. While rejecting the idea of a legal presumption of equal access, they called for more rigorous enforcement of court orders and a change in the law to specifically consider the importance of sustaining the relationship between the child and the non-resident parent in contested cases. They also emphasized the need for professional mediation and negotiation to be prioritized over court involvement in resolving these sensitive family matters.'
fe.default(article_text,summ)

(0.0, 'bad')

Calculate scores for whole dataset

In [15]:
for i,datum in enumerate(data):
    print('{}/{}'.format(i, len(data)))
    article=datum['article_text']
    LLM=datum['text-davinci-002_summary']
    datum['faithfullness'], datum['faithfullness_binary'] = fe.default(article,LLM) # Remove 'datum['faithfullness_binary']' if you dont have Faithfulness_test.json 
    print("score",datum['faithfullness'])

0/599
score 0.5
1/599
score 0.8
2/599
score 0.7777777777777778
3/599
score 0.6666666666666666
4/599
score 0.2
5/599
score 0.75
6/599
score 0.6
7/599
score 0.0
8/599


KeyboardInterrupt: 

In [7]:
save_json(data,r'D:\Laptop_Projects\LLMEval-1\preprocess\experiments\data\Faithfulness_binary.json')